# Demo: Serenity VaR Calculation API

In addition to supporting risk attribution, the latest version of Serenity also supports running VaR calculations
and backtesting those VaR models with your portfolio. This notebook will show you how to select a VaR model and
use it to compute Value at Risk (VaR) for a portfolio constructed from the Serenity security master.

In [ ]:
%%capture --no-stderr --no-display
%load_ext autoreload
%autoreload 2
%run -i init_demo.py

To run a VaR calculation the first thing we need is a portfolio:

In [ ]:
asset_master = api.refdata().load_asset_master()

portfolio_raw = {
    'ADA': 1000000,
    'BTC': 100,
    'ETH': 1000,
    'XRP': 2000000,
    'ALGO': 1500000,
    'SOL': 10000,
    'DOT': 50000
}
portfolio = asset_master.create_portfolio(portfolio_raw, symbology='NATIVE')

The next thing we need is a model configuration. IMPORTANT: this feature is not yet supported in production, but we are including it here because this is the pattern you will need to follow going forward:

In [ ]:
from serenity_sdk.types import CalculationContext

model_short_name = 'risk.var.parametric.normal'  
model_meta = api.model().load_model_metadata(datetime.date.today())
model_config_id = model_meta.get_model_configuration_id(model_short_name)

Now that we have it we can create the `CalculationContext`, the same as with risk attribution. We will set an as-of-date and pass our parametric VaR model configuration ID. We then run VaR for a given date:

In [ ]:
from datetime import date

# construct the input parameters for VaR
ctx = CalculationContext(as_of_date=date(2021, 7, 1), model_config_id=model_config_id)

# run the VaR calc
result = api.risk().compute_var(ctx, portfolio)
result

We can also run a backtest:

In [ ]:
result = api.risk().compute_var_backtest(ctx, portfolio, date(2021, 7, 1), date(2022, 7, 1))

In [ ]:
rows = [{
    'runDate': result.run_date,
    'baseline': result.baseline,
    'varAbsolute': result.quantiles[0].var_absolute,
    'varRelative': result.quantiles[0].var_relative
} for result in result.results]
backtest_df = pd.DataFrame(rows)
backtest_df.set_index('runDate', inplace=True)

loss_fmt = lambda val: f'${val:,.2f}' if val >= 0 else f'(${abs(val):,.2f})'
pct_fmt = lambda val: f'({abs(val):,.1%})'
backtest_df.style.format({
  'baseline': loss_fmt,
  'varAbsolute': loss_fmt,
  'varRelative': pct_fmt
}).applymap(lambda val: 'color: red', subset=['varRelative'])

And we can identify the breach dates:

In [ ]:
rows = [{
    'breachDate': breach.breach_date.date(),
    'portfolioLossAbsolute': breach.portfolio_loss_absolute,
    'portfolioLossRelative': breach.portfolio_loss_relative,
    'varLevelAbsolute': breach.quantiles[0].var_absolute,
    'varLevelRelative': breach.quantiles[0].var_relative,
} for breach in result.breaches]
backtest_df = pd.DataFrame(rows)
backtest_df.set_index('breachDate', inplace=True)
backtest_df.style.format({
  'portfolioLossAbsolute': loss_fmt,
  'varLevelAbsolute': loss_fmt,
  'portfolioLossRelative': pct_fmt,
  'varLevelRelative': pct_fmt
}).applymap(lambda val: 'color: red', subset=['portfolioLossAbsolute', 'portfolioLossRelative', 'varLevelAbsolute', 'varLevelRelative'])